# 08 — Comparação Entre Modelos e Análise Final

Projeto: Classificação de Pneumonia em Raio-X  
Liga Acadêmica de Inteligência Artificial — UFPE

## Objetivo

Este notebook consolida os resultados de todos os experimentos,
gerando visualizações e análises para o relatório técnico-científico.

## Experimentos

| Experimento | Modelo | Augmentation | Class Weight |
|---|---|---|---|
| Baseline | ResNet18 | Leve | Não |
| H1 | DenseNet121 | Leve | Não |
| H2 | ResNet18 | Forte | Não |
| H3 | ResNet18 | Leve | Sim |

**Modelo recomendado: H3 (Class Weight)** — superior em relevância clínica,
reduzindo Falsos Negativos de 16 (Baseline) para 5 (redução de 69%).


## 1. Setup

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import pandas as pd

NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

metrics_dir = os.path.join(PROJECT_ROOT, "outputs", "metrics")
figures_dir = os.path.join(PROJECT_ROOT, "outputs", "figures")
os.makedirs(figures_dir, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Figures dir: ", figures_dir)

## 2. Carregamento das Métricas

Cada arquivo `.pkl` foi gerado pelo respectivo notebook de treinamento
e contém o `history` completo (loss, AUC, F1, Recall, Precision por época).

In [ ]:
def load_metrics(path):
    with open(path, "rb") as f:
        return pickle.load(f)

baseline = load_metrics(os.path.join(metrics_dir, "resnet18_light_noCW.pkl"))
h1       = load_metrics(os.path.join(metrics_dir, "densenet121_light_noCW.pkl"))
h2       = load_metrics(os.path.join(metrics_dir, "resnet18_strong_noCW.pkl"))
h3       = load_metrics(os.path.join(metrics_dir, "resnet18_light_CW.pkl"))

experiments = {
    "Baseline (ResNet18)": baseline,
    "H1 — DenseNet121":    h1,
    "H2 — Strong Aug":     h2,
    "H3 — Class Weight":   h3,
}

# Paleta de cores consistente em todos os plots
COLORS = {
    "Baseline (ResNet18)": "#5B8DB8",
    "H1 — DenseNet121":    "#E8A838",
    "H2 — Strong Aug":     "#7DBE7D",
    "H3 — Class Weight":   "#C0504D",
}

print("Métricas carregadas:", list(experiments.keys()))

## 3. Tabela Comparativa Final

In [ ]:
summary_data = {
    "Experimento":     ["Baseline", "H1 — DenseNet121", "H2 — Strong Aug", "H3 — Class Weight"],
    "Best AUC":        [0.999029,   0.998939,            0.998952,           0.999154],
    "F1":              [0.988453,   0.987269,            0.984901,           0.993127],
    "Recall":          [0.981651,   0.978211,            0.972477,           0.994266],
    "Precision":       [0.995349,   0.996495,            0.997647,           0.991991],
    "Accuracy":        [0.982441,   0.980685,            0.977173,           0.989464],
    "Sensitivity":     [0.981651,   0.978211,            0.972477,           0.994266],
    "Specificity":     [0.985019,   0.988764,            0.992509,           0.973783],
    "False Negatives": [16,         19,                  24,                 5],
    "Best Epoch":      [4,          9,                   9,                  5],
}

df_summary = pd.DataFrame(summary_data).set_index("Experimento")
print(df_summary.to_string())

## 4. Evolução do AUC por Época

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for name, data in experiments.items():
    auc_curve = data["history"]["val_auc"]
    epochs = range(1, len(auc_curve) + 1)
    ax.plot(epochs, auc_curve, marker="o", label=name,
            color=COLORS[name], linewidth=2, markersize=5)

ax.set_title("Evolução do ROC-AUC na Validação por Época",
             fontsize=13, fontweight="bold", pad=12)
ax.set_xlabel("Época", fontsize=11)
ax.set_ylabel("ROC-AUC", fontsize=11)
ax.set_xticks(range(1, 11))
ax.set_ylim(0.995, 1.001)
ax.legend(fontsize=10, loc="lower right")
ax.grid(True, linestyle="--", alpha=0.5)
ax.spines[["top", "right"]].set_visible(False)

plt.tight_layout()
save_path = os.path.join(figures_dir, "auc_por_epoca.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 5. Curvas de Loss — Treino vs Validação

Divergência entre train e val loss indica sobreajuste.

In [ ]:
best_epochs = {"Baseline (ResNet18)": 4, "H1 — DenseNet121": 9,
               "H2 — Strong Aug": 9, "H3 — Class Weight": 5}

fig, axes = plt.subplots(2, 2, figsize=(13, 9))
axes = axes.flatten()

for i, (name, data) in enumerate(experiments.items()):
    h = data["history"]
    epochs = range(1, len(h["train_loss"]) + 1)
    best_ep = best_epochs[name]

    axes[i].plot(epochs, h["train_loss"], label="Train Loss",
                 color=COLORS[name], linewidth=2)
    axes[i].plot(epochs, h["val_loss"], label="Val Loss",
                 color=COLORS[name], linewidth=2, linestyle="--", alpha=0.7)
    axes[i].axvline(best_ep, color="gray", linestyle=":",
                    alpha=0.8, label=f"Melhor época ({best_ep})")

    axes[i].set_title(name, fontsize=11, fontweight="bold")
    axes[i].set_xlabel("Época", fontsize=9)
    axes[i].set_ylabel("Loss", fontsize=9)
    axes[i].set_xticks(range(1, 11))
    axes[i].legend(fontsize=8)
    axes[i].grid(True, linestyle="--", alpha=0.4)
    axes[i].spines[["top", "right"]].set_visible(False)

fig.suptitle("Curvas de Loss — Treino vs Validação",
             fontsize=13, fontweight="bold", y=1.01)
plt.tight_layout()
save_path = os.path.join(figures_dir, "loss_curves.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 6. Comparação de Métricas Finais

In [ ]:
metrics_to_plot = ["Best AUC", "F1", "Recall", "Precision", "Accuracy"]
labels = ["Baseline", "H1\nDenseNet121", "H2\nStrong Aug", "H3\nClass Weight"]
colors = list(COLORS.values())

fig, axes = plt.subplots(1, len(metrics_to_plot), figsize=(16, 5))

for j, metric in enumerate(metrics_to_plot):
    values = df_summary[metric].values
    bars = axes[j].bar(labels, values, color=colors, edgecolor="white", linewidth=0.5)
    bars[3].set_edgecolor("black")
    bars[3].set_linewidth(2)

    for bar, val in zip(bars, values):
        axes[j].text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.0002,
            f"{val:.4f}",
            ha="center", va="bottom", fontsize=7.5, rotation=45
        )

    axes[j].set_ylim(min(values) - 0.005, max(values) + 0.008)
    axes[j].set_title(metric, fontsize=10, fontweight="bold")
    axes[j].set_ylabel("Score" if j == 0 else "", fontsize=9)
    axes[j].tick_params(axis="x", labelsize=8)
    axes[j].grid(True, axis="y", linestyle="--", alpha=0.4)
    axes[j].spines[["top", "right"]].set_visible(False)

fig.suptitle("Comparação de Métricas de Desempenho — Todos os Experimentos",
             fontsize=13, fontweight="bold", y=1.02)

patch = mpatches.Patch(facecolor=COLORS["H3 — Class Weight"], edgecolor="black",
                       linewidth=2, label="H3 — Class Weight (modelo recomendado)")
fig.legend(handles=[patch], loc="lower center", fontsize=9, bbox_to_anchor=(0.5, -0.05))

plt.tight_layout()
save_path = os.path.join(figures_dir, "metricas_comparacao.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 7. Análise Clínica — Falsos Negativos e Tradeoff

Em diagnóstico médico, o custo de um FN é assimétrico:
uma pneumonia não detectada pode evoluir para complicações graves.
O H3 reduz os FNs de 16 (Baseline) para 5 — redução de 69%.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# --- Plot 1: Falsos Negativos ---
exp_labels = ["Baseline", "H1\nDenseNet121", "H2\nStrong Aug", "H3\nClass Weight"]
fn_values  = [16, 19, 24, 5]
bar_colors = list(COLORS.values())

bars = axes[0].bar(exp_labels, fn_values, color=bar_colors, edgecolor="white")
bars[3].set_edgecolor("black")
bars[3].set_linewidth(2)

for bar, val in zip(bars, fn_values):
    axes[0].text(bar.get_x() + bar.get_width() / 2,
                 bar.get_height() + 0.3, str(val),
                 ha="center", fontsize=12, fontweight="bold")

axes[0].annotate(
    "−69% FNs\nvs Baseline",
    xy=(3, 5), xytext=(2.1, 20),
    arrowprops=dict(arrowstyle="->", color="black"),
    fontsize=10, fontweight="bold"
)
axes[0].set_title("Falsos Negativos por Experimento\n(menor é melhor clinicamente)",
                  fontsize=11, fontweight="bold")
axes[0].set_ylabel("Número de FNs", fontsize=10)
axes[0].set_ylim(0, 30)
axes[0].grid(True, axis="y", linestyle="--", alpha=0.4)
axes[0].spines[["top", "right"]].set_visible(False)

# --- Plot 2: Sensitivity vs Specificity ---
sensitivity = [0.981651, 0.978211, 0.972477, 0.994266]
specificity = [0.985019, 0.988764, 0.992509, 0.973783]
exp_short   = ["Baseline", "H1", "H2", "H3"]

for i, (s, sp, nm, c) in enumerate(zip(sensitivity, specificity, exp_short, bar_colors)):
    axes[1].scatter(sp, s, color=c, s=200, zorder=5,
                    edgecolors="black" if i == 3 else "white",
                    linewidths=2 if i == 3 else 0.5)
    axes[1].annotate(nm, (sp, s), textcoords="offset points",
                     xytext=(8, 4), fontsize=10,
                     fontweight="bold" if i == 3 else "normal", color=c)

axes[1].set_xlabel("Specificity (TNR)", fontsize=11)
axes[1].set_ylabel("Sensitivity (Recall / TPR)", fontsize=11)
axes[1].set_title("Tradeoff Clínico: Sensitivity vs Specificity",
                  fontsize=11, fontweight="bold")
axes[1].set_xlim(0.965, 1.002)
axes[1].set_ylim(0.965, 1.002)
axes[1].grid(True, linestyle="--", alpha=0.4)
axes[1].spines[["top", "right"]].set_visible(False)
axes[1].text(0.997, 0.998, "Ideal", fontsize=8, color="gray",
             ha="right", style="italic")

plt.tight_layout()
save_path = os.path.join(figures_dir, "analise_clinica.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 8. Curvas ROC — Melhor Época de Cada Experimento

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

for name, data in experiments.items():
    h = data["history"]
    best_ep = int(np.argmax(h["val_auc"]))
    fpr = h["val_fpr"][best_ep]
    tpr = h["val_tpr"][best_ep]
    auc = h["val_auc"][best_ep]
    lw  = 2.5 if "Class Weight" in name else 1.5

    ax.plot(fpr, tpr, label=f"{name} (AUC={auc:.4f})",
            color=COLORS[name], linewidth=lw)

ax.plot([0, 1], [0, 1], "k--", linewidth=1, label="Random (AUC=0.5)")
ax.set_xlabel("False Positive Rate (1 - Specificity)", fontsize=11)
ax.set_ylabel("True Positive Rate (Sensitivity)", fontsize=11)
ax.set_title("Curvas ROC — Melhor Época por Experimento",
             fontsize=13, fontweight="bold")
ax.legend(fontsize=10, loc="lower right")
ax.grid(True, linestyle="--", alpha=0.4)
ax.spines[["top", "right"]].set_visible(False)

plt.tight_layout()
save_path = os.path.join(figures_dir, "roc_curves.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 9. Evolução do Recall por Época

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

for name, data in experiments.items():
    recall_curve = data["history"]["val_recall"]
    epochs = range(1, len(recall_curve) + 1)
    lw = 2.5 if "Class Weight" in name else 1.8
    ax.plot(epochs, recall_curve, marker="o", label=name,
            color=COLORS[name], linewidth=lw, markersize=5)

ax.set_title("Evolução do Recall (Sensitivity) na Validação por Época",
             fontsize=13, fontweight="bold", pad=12)
ax.set_xlabel("Época", fontsize=11)
ax.set_ylabel("Recall", fontsize=11)
ax.set_xticks(range(1, 11))
ax.set_ylim(0.93, 1.01)
ax.legend(fontsize=10)
ax.grid(True, linestyle="--", alpha=0.5)
ax.spines[["top", "right"]].set_visible(False)

# Anotação H3
h3_recall = h3["history"]["val_recall"]
best_h3   = max(h3_recall)
ep_h3     = h3_recall.index(best_h3) + 1
ax.annotate(
    f"H3 peak: {best_h3:.4f}",
    xy=(ep_h3, best_h3),
    xytext=(ep_h3 + 0.6, best_h3 - 0.01),
    arrowprops=dict(arrowstyle="->", color=COLORS["H3 — Class Weight"]),
    fontsize=9, color=COLORS["H3 — Class Weight"], fontweight="bold"
)

plt.tight_layout()
save_path = os.path.join(figures_dir, "recall_por_epoca.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 10. Análise das Hipóteses

In [ ]:
print("=" * 65)
print("ANÁLISE DAS HIPÓTESES EXPERIMENTAIS")
print("=" * 65)

print("""
HIPÓTESE 1 — Arquitetura: DenseNet121 vs ResNet18
--------------------------------------------------
Resultado: NÃO CONFIRMADA

A DenseNet121, apesar de sua arquitetura distinta com conexões
densas (dense connections), não superou a ResNet18 em nenhuma
métrica relevante. O AUC foi ligeiramente inferior (0.9989 vs
0.9990) e os FNs aumentaram de 16 para 19.

Nota: A hipótese foi reformulada — a DenseNet121 não é apenas
"mais profunda", mas representa um paradigma arquitetural
diferente (reutilização de features via concatenação vs
adição residual na ResNet). Para este dataset (~4000 amostras),
a ResNet18 oferece melhor custo-benefício.
""")

print("""
HIPÓTESE 2 — Data Augmentation Intenso
--------------------------------------------------
Resultado: NÃO CONFIRMADA

Transformações mais agressivas (rotação ±15°, affine, jitter)
não melhoraram o desempenho. Recall e FNs pioraram (24 FNs vs
16 do Baseline). A Specificity melhorou (0.9925), indicando
que o modelo ficou mais conservador — evita FPs mas erra mais
FNs. Para imagens médicas, augmentations geométricas agressivas
podem distorcer padrões patológicos relevantes.
""")

print("""
HIPÓTESE 3 — Ponderação de Classes (Class Weight)
--------------------------------------------------
Resultado: CONFIRMADA ✓

  • Melhor AUC:   0.9992 (marginalmente superior)
  • Maior Recall: 0.9943 (+1.3pp vs Baseline)
  • Maior F1:     0.9931
  • Menor FNs:    5 (vs 16 do Baseline, redução de 69%)

Tradeoff esperado e clinicamente aceitável: leve queda de
Specificity (0.9738 vs 0.9850). Convergência rápida (melhor
época: 5) sugere que o class weighting fornece sinal de
treinamento mais informativo desde o início.
""")

print("=" * 65)

## 11. Dashboard Final — Figura para o Relatório

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 3, figure=fig, hspace=0.45, wspace=0.35)

ax1 = fig.add_subplot(gs[0, 0])  # AUC por época
ax2 = fig.add_subplot(gs[0, 1])  # Recall por época
ax3 = fig.add_subplot(gs[0, 2])  # FNs
ax4 = fig.add_subplot(gs[1, 0])  # Sensitivity vs Specificity
ax5 = fig.add_subplot(gs[1, 1:]) # Tabela de métricas

# AUC por época
for name, data in experiments.items():
    auc_c = data["history"]["val_auc"]
    ax1.plot(range(1, len(auc_c)+1), auc_c, marker="o",
             color=COLORS[name], linewidth=1.8, markersize=4, label=name)
ax1.set_title("AUC por Época", fontsize=10, fontweight="bold")
ax1.set_xlabel("Época", fontsize=8); ax1.set_ylabel("AUC", fontsize=8)
ax1.set_ylim(0.995, 1.001); ax1.set_xticks(range(1, 11))
ax1.grid(True, linestyle="--", alpha=0.4); ax1.tick_params(labelsize=7)
ax1.spines[["top","right"]].set_visible(False)

# Recall por época
for name, data in experiments.items():
    rec = data["history"]["val_recall"]
    lw  = 2.5 if "Class Weight" in name else 1.5
    ax2.plot(range(1, len(rec)+1), rec, marker="o",
             color=COLORS[name], linewidth=lw, markersize=4)
ax2.set_title("Recall por Época", fontsize=10, fontweight="bold")
ax2.set_xlabel("Época", fontsize=8); ax2.set_ylabel("Recall", fontsize=8)
ax2.set_ylim(0.92, 1.01); ax2.set_xticks(range(1, 11))
ax2.grid(True, linestyle="--", alpha=0.4); ax2.tick_params(labelsize=7)
ax2.spines[["top","right"]].set_visible(False)

# FNs
fn_vals     = [16, 19, 24, 5]
short_lbls  = ["Baseline", "H1", "H2", "H3★"]
b = ax3.bar(short_lbls, fn_vals, color=list(COLORS.values()), edgecolor="white")
b[3].set_edgecolor("black"); b[3].set_linewidth(2)
for bar, v in zip(b, fn_vals):
    ax3.text(bar.get_x()+bar.get_width()/2, v+0.3, str(v),
             ha="center", fontsize=10, fontweight="bold")
ax3.annotate("−69%", xy=(3, 5), xytext=(2.1, 18),
             arrowprops=dict(arrowstyle="->", color="black"),
             fontsize=9, fontweight="bold")
ax3.set_title("Falsos Negativos (FN)", fontsize=10, fontweight="bold")
ax3.set_ylabel("Contagem", fontsize=8); ax3.set_ylim(0, 30)
ax3.grid(True, axis="y", linestyle="--", alpha=0.4); ax3.tick_params(labelsize=8)
ax3.spines[["top","right"]].set_visible(False)

# Sensitivity vs Specificity
sens = [0.981651, 0.978211, 0.972477, 0.994266]
spec = [0.985019, 0.988764, 0.992509, 0.973783]
sn   = ["Baseline", "H1", "H2", "H3"]
for i, (s, sp, nm, c) in enumerate(zip(sens, spec, sn, list(COLORS.values()))):
    ax4.scatter(sp, s, color=c, s=150, zorder=5,
                edgecolors="black" if i==3 else "white",
                linewidths=2 if i==3 else 0.5)
    ax4.annotate(nm, (sp, s), xytext=(5, 3), textcoords="offset points",
                 fontsize=8, fontweight="bold" if i==3 else "normal", color=c)
ax4.set_xlabel("Specificity", fontsize=8); ax4.set_ylabel("Sensitivity", fontsize=8)
ax4.set_title("Sensitivity vs Specificity", fontsize=10, fontweight="bold")
ax4.set_xlim(0.965, 1.002); ax4.set_ylim(0.965, 1.002)
ax4.grid(True, linestyle="--", alpha=0.4); ax4.tick_params(labelsize=7)
ax4.spines[["top","right"]].set_visible(False)

# Tabela de métricas
ax5.axis("off")
table_data = [
    ["Baseline",    "0.9990", "0.9885", "0.9817", "0.9953", "16"],
    ["H1 DenseNet", "0.9989", "0.9873", "0.9782", "0.9965", "19"],
    ["H2 Strong",   "0.9990", "0.9849", "0.9725", "0.9976", "24"],
    ["H3 CW ★",     "0.9992", "0.9931", "0.9943", "0.9920",  "5"],
]
col_labels = ["Modelo", "AUC", "F1", "Recall", "Precision", "FNs"]
table = ax5.table(cellText=table_data, colLabels=col_labels,
                  loc="center", cellLoc="center")
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
for col in range(len(col_labels)):
    table[4, col].set_facecolor("#FADADD")
    table[4, col].set_text_props(fontweight="bold")
    table[0, col].set_facecolor("#2C3E50")
    table[0, col].set_text_props(color="white", fontweight="bold")
ax5.set_title("Resumo das Métricas Finais  (★ = modelo recomendado)",
              fontsize=10, fontweight="bold", pad=20)

fig.suptitle("Comparação de Experimentos — Classificação de Pneumonia em Raio-X",
             fontsize=14, fontweight="bold", y=1.01)

handles = [mpatches.Patch(color=c, label=n) for n, c in COLORS.items()]
fig.legend(handles=handles, loc="lower center", ncol=4,
           fontsize=8, bbox_to_anchor=(0.5, -0.04))

save_path = os.path.join(figures_dir, "dashboard_comparacao.png")
plt.savefig(save_path, dpi=150, bbox_inches="tight")
plt.show()
print("Salvo em:", save_path)

## 12. Resumo Executivo

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║              RESUMO EXECUTIVO — MODELO RECOMENDADO           ║
╠══════════════════════════════════════════════════════════════╣
║  Modelo:  ResNet18 + Class Weighting (H3)                    ║
║  Arquivo: resnet18_light_CW.pt                               ║
╠══════════════════════════════════════════════════════════════╣
║  AUC:        0.9992  (melhor entre todos os experimentos)    ║
║  Recall:     0.9943  (+1.3pp vs Baseline)                    ║
║  F1-Score:   0.9931  (melhor entre todos)                    ║
║  FNs:        5       (vs 16 do Baseline, redução de 69%)     ║
╠══════════════════════════════════════════════════════════════╣
║  Justificativa clínica:                                      ║
║  Em diagnóstico de pneumonia, o custo assimétrico entre      ║
║  FN (pneumonia não detectada) e FP (alarme falso) justifica  ║
║  a escolha de um modelo com maior Sensitivity. O H3 é o      ║
║  único modelo explicitamente desenhado para este tradeoff.   ║
╚══════════════════════════════════════════════════════════════╝
""")

print("Figuras geradas em:", figures_dir)
for fig_name in ["auc_por_epoca.png", "loss_curves.png", "metricas_comparacao.png",
                 "analise_clinica.png", "roc_curves.png", "recall_por_epoca.png",
                 "dashboard_comparacao.png"]:
    print(" •", fig_name)